In [2]:
import pandas as pd

In [ ]:
expr_unnamed_df = pd.read_csv(
    "../results/predictions/trained_S1_expr/test_metrics_summary.csv"
).round(3)
expr_df = expr_unnamed_df.rename(columns={"Unnamed: 0": "metric"})
expr_df["Dataset"] = "GE"
expr_df

,metric,MAE,RMSE,MDAE,PCC,Dataset
0,mean,0.101,0.142,0.076,0.729,GE
1,std,0.057,0.076,0.043,0.230,GE


In [ ]:
flux_unnamed_df = pd.read_csv(
    "../results/predictions/trained_S3_flux/test_metrics_summary.csv"
).round(3)
flux_df = flux_unnamed_df.rename(columns={"Unnamed: 0": "metric"})
flux_df["Dataset"] = "MF"
# num_cols = flux_df.select_dtypes(include="number").columns
# flux_df[num_cols] = flux_df[num_cols].applymap(lambda x: f"{x:.4g}")
flux_df

,metric,MAE,RMSE,MDAE,PCC,Dataset
0,mean,1.212383e+19,1.684216e+20,0.161,-0.042,MF
1,std,1.742623e+19,2.420912e+20,0.087,0.059,MF


In [ ]:
prot_unnamed_df = pd.read_csv(
    "../results/predictions/trained_S2_prot/test_metrics_summary.csv"
).round(3)
prot_df = prot_unnamed_df.rename(columns={"Unnamed: 0": "metric"})
prot_df["Dataset"] = "PA"
prot_df

,metric,MAE,RMSE,MDAE,PCC,Dataset
0,mean,0.104,0.161,0.074,0.454,PA
1,std,0.012,0.017,0.007,0.180,PA


In [6]:
df = pd.concat([expr_df, flux_df, prot_df, ], ignore_index=True)
df

,metric,MAE,RMSE,MDAE,PCC,Dataset
0,mean,1.010000e-01,1.420000e-01,0.076,0.729,GE
1,std,5.700000e-02,7.600000e-02,0.043,0.230,GE
2,mean,1.212383e+19,1.684216e+20,0.161,-0.042,MF
3,std,1.742623e+19,2.420912e+20,0.087,0.059,MF
4,mean,1.040000e-01,1.610000e-01,0.074,0.454,PA
5,std,1.200000e-02,1.700000e-02,0.007,0.180,PA


In [ ]:
metrics = ["MAE", "RMSE", "MDAE", "PCC"]

# build display table grouped by Dataset
display_df = pd.DataFrame()


for dataset in df["Dataset"].unique():
    subset = df[df["Dataset"] == dataset]
    mean_row = subset[subset["metric"] == "mean"][metrics].iloc[0]
    std_row = subset[subset["metric"] == "std"][metrics].iloc[0]

    combined = mean_row.map("{:.4g}".format) + " ± " + std_row.map("{:.4g}".format)
    combined["Dataset"] = dataset

    display_df = pd.concat([display_df, pd.DataFrame([combined])], ignore_index=True)

display_df

,MAE,RMSE,MDAE,PCC,Dataset
0,0.101 ± 0.057,0.142 ± 0.076,0.076 ± 0.043,0.729 ± 0.23,GE
1,1.212e+19 ± 1.743e+19,1.684e+20 ± 2.421e+20,0.161 ± 0.087,-0.042 ± 0.059,MF
2,0.104 ± 0.012,0.161 ± 0.017,0.074 ± 0.007,0.454 ± 0.18,PA


In [12]:
means_only = df[df["metric"] == "mean"].set_index("Dataset")[metrics]

is_higher_better = {"PCC": True}

best_idx = {}
second_idx = {}

for metric in metrics:
    vals = means_only[metric]
    if is_higher_better.get(metric, False):
        rank = vals.rank(method="min", ascending=False)
    else:
        rank = vals.rank(method="min", ascending=True)
    best_idx[metric] = rank[rank == 1].index
    second_idx[metric] = rank[rank == 2].index


In [14]:
formatted_df = display_df.copy()

for metric in metrics:
    for i, row in formatted_df.iterrows():
        dataset = row["Dataset"]
        val = row[metric]
        if dataset in best_idx[metric]:
            formatted_df.at[i, metric] = r"\textbf{" + val + "}"
        elif dataset in second_idx[metric]:
            formatted_df.at[i, metric] = r"\underline{" + val + "}"